# <a href="https://miptstats.github.io/courses/ad_mipt.html">Phystech@DataScience</a>
## Домашнее задание 3

**Правила, <font color="red">прочитайте внимательно</font>:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_pds_bot`. Для начала работы с ботом каждый раз отправляйте `/start`. **Работы, присланные иным способом, не принимаются.**
* Дедлайн см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно ноутбук в формате `ipynb`.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания будут сдавать устный зачет.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* **Код из рассказанных на занятиях ноутбуков можно использовать без ограничений.**

**Правила оформления теоретических задач:**

* Решения необходимо прислать одним из следующих способов:
  * фотографией в правильной ориентации, где все четко видно, а почерк разборчив,
    * отправив ее как файл боту вместе с ноутбуком *или*
    * вставив ее в ноутбук посредством `Edit -> Insert Image` (<font color="red">фото, вставленные ссылкой, не принимаются</font>);
  * в виде $\LaTeX$ в markdown-ячейках.
* Решения не проверяются, если какое-то требование не выполнено. Особенно внимательно все проверьте в случае выбора второго пункта (вставки фото в ноутбук). <font color="red"><b>Неправильно вставленные фотографии могут не передаться при отправке.</b></font> Для проверки попробуйте переместить `ipynb` в другую папку и открыть его там.
* В решениях поясняйте, чем вы пользуетесь, хотя бы кратко. Например, если пользуетесь независимостью, то достаточно подписи вида "*X и Y незав.*"
* Решение, в котором есть только ответ, и отсутствуют вычисления, оценивается в 0 баллов.

**Баллы за задание:**

* Отрисовка изображений &mdash; 10 баллов
* Построение сети ResNet &mdash; 80 баллов
* Transfer Learning &mdash; 30 баллов





-----

In [ ]:
# Bot check

# HW_ID: phds_hw3
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: not final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную.
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

## Сверточные сети

В этой домашней работе вам предстоит улучшить сверточную сеть для классификации изображений.

#### Биология
Необходимо классифицировать изображения МРТ головного мозга из датасета [**"Brain Tumor Classification (MRI)"**](https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri) и определить вид опухоли.

#### Физика
Необходимо классифицировать изображения солнечного затмения из датасета [**"Solar Eclipse Classification"**](https://www.kaggle.com/datasets/tshr147/solar-eclipse-classification) по степени: частичное, полное и кольцевое.

## Задача 1.

**Пожалуйста, ПРОЧИТАЙТЕ ВНИМАТЕЛЬНО то, что написано ниже, там изложены требования к вашей работе и полезные советы!**


**Требование к работе**

* **Запрещено** использовать тестовые данные где-либо за исключением вычисления финальной оценки качества. Подсказка &mdash; распределение данных на тесте такое же как в тестовых данных.


### Советы

#### Архитектура нейросети
* В отличие от семинара в данном датасете могут встретиться картинки разных размеров. Эту проблему можно решить двумя способами:
  - Используя `torchvision.transforms.Resize` можно привести картинки к единому размеру. Если вы решите использовать этот способ, стоит посмотреть, какого в принципе размера встречаются картинки, чтобы не сжать их слишком сильно. Для картинок одного размера можно обучить бейзлайн в виде полносвязной сети.
  - Учесть переменный размер картинки в архитектуре сети. Общий принцип здесь такой: можно использовать свертки с нужным `padding`, чтобы не иметь проблем из-за уменьшения размеров картинки из-за сверток, последовательно применяя сверточные слои и пуллинги, нужно увеличивать количество каналов одновременно с уменьшением размера картинок (из-за пуллинга), а в конце, получив картинку размера (n_channels, nx, ny), оставить вектор размера (n_channels) (n_channels будет одинаковым для всех картинок, поскольку зависит от архитектуры сети!). Сделать это можно усреднением по пространственным картам [torch.nn.AdaptiveAvgPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html).
* Попробуйте разные размеры фильтров, страйдинг, паддинг
* Также можно попробовать разные активации: `tanh`, `leaky relu` и другие.

#### Процесс обучения
* Воспользуйтесь GPU google colab или любой другой GPU, которая у вас есть.
* Для сокращения вычислительной сложности можно поэксперементировать с параметром `stride`. Кроме того можете попробовать разные виды Poooling-ов.
* Помните, что некоторым нейросетям требуется $10$ эпох, чтобы сойтись, а некоторым – $500$. Большие нейросети дольше обучаются.
* Если вы достигли какого-то порога на валидации лучше подождать примерно 10 эпох перед тем как останавливать обучение.

#### И главное:
* Рисуйте кривые обучения: loss и метрика качества (лучше использовать F1-меру) для обучения и валидации.

In [ ]:
! pip install torchinfo

from torchinfo import summary

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil

import torch
from torch import nn
import torchvision
from torchvision import transforms
from sklearn.metrics import f1_score
from sklearn.utils.random import sample_without_replacement
from IPython.display import Image, clear_output
from collections import defaultdict
from torch.optim import lr_scheduler
from matplotlib.animation import FuncAnimation, ImageMagickFileWriter
import time

%matplotlib inline

In [ ]:
torch.cuda.is_available()

### Для Google Colab
Чтобы не грузить данные каждый раз в колаб при его отключении, а данные сюда грузятся небыстро, будет лучше всего поступить следующим образом.
* Загрузите архив на диск.
* Примонтируйте ваш диск к данному ноутбуку с помощью кода ниже


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

* В панели слева ("Файлы") откройте папку `drive/MyDrive/..` и найдите архив с файлом на диске
* Кликните по файлу и нажмите на кнопку "Скопировать путь"

Теперь вы можете обратиться к данным, используя скопированный путь

## Предобработка данных

In [ ]:
# Путь до архива с данными (пример)
ZIP_PATH = "/home/e-latypova/workspace/archive_bio.zip"
# Путь для папки с данными (пример)
DATA_PATH = "/home/e-latypova/workspace/archive_bio"

Разархивируем данные на диске.

In [ ]:
! rm -rf $DATA_PATH

In [ ]:
! unzip $ZIP_PATH -d  $DATA_PATH

In [ ]:
! ls $DATA_PATH

In [ ]:
output = os.popen(f'ls {DATA_PATH}', 'r').read()
output = output.replace('\x1b[34m', '')
output = output.replace('\x1b[m', '')
test_path = output.split('\n')[0]
train_path = output.split('\n')[1]

### Подготовка датасетов

In [ ]:
# Папка с изображениями для тренировки
TRAIN_DIR = os.path.join(DATA_PATH, train_path)
# Папка с изображениями для валидации
VAL_DIR = os.path.join(DATA_PATH, "val")

# Папка с изображениями для теста
TEST_DIR = os.path.join(DATA_PATH, test_path)

In [ ]:
! rm -rf $VAL_DIR
os.makedirs(VAL_DIR, exist_ok=True)

# Считываем названия директорий
DIR_LIST = {i:name for i, name in enumerate(os.listdir(TRAIN_DIR))}
# Доля изображений в валидации
VAL_FRAC = 0.3

# Создаем директорию с валидационной выборкой для каждого класса
for dir in DIR_LIST.values():
    os.makedirs(os.path.join(VAL_DIR, dir), exist_ok=True)

    # Считываем выборку изображений
    dir_path = os.path.join(TRAIN_DIR, dir)

    # Сортируем изображения для детерминированнсти
    images_filename = sorted(os.listdir(dir_path))

    # Выбираем случайные изображения из выборки для валидции, с установленным random_state
    num_images = len(images_filename)
    num_val = int(num_images * VAL_FRAC)
    indices = sample_without_replacement(num_images, num_val, random_state=42)
    val_images = np.take(images_filename, indices)

    print(f'{dir} | train images = {num_images - num_val} | val images = {num_val}')

    # Сохраняем валидационную выборку
    for image_filename in val_images:
        source = os.path.join(TRAIN_DIR, dir, image_filename)
        destination = os.path.join(VAL_DIR, dir, image_filename)
        shutil.copy(source, destination)
        os.remove(source)

In [ ]:
!ls $TRAIN_DIR

In [ ]:
!ls $VAL_DIR

Отрисуйте по 3 изображения для каждого класса.

In [ ]:
#ваш код

### I. Построение сети ResNet

В первой части задания вам предстоит имплементировать сеть ResNet.

#### Архитектура сети

<img src="resblock.png" width="400" align="right" >

Артихектура сети выглядит следующим образом

- В начале сети применяется сверточный слой с большим размером ядра (можно взять 5-7) и пулинг для уменьшения размерности входного изображения.
- Далее следует какое-то количество ResidualBlock'ов. Блоки бывают двух типов: не меняющие пространственную и канальную размерности и сжимающие изображение одновременно с увеличением количества каналов. По мере продвижения в глубину сети количество каналов должно увеличиваться.
- Перед предсказанием класса применяется Global Average Pooling для получения вектора фиксированной размерности, равной количеству каналов в конце сети.
- Предсказание выполняется одним линейным слоем.

Посмотрим как выглядит эффективный ResidualBlock, использованный в оригинальной статье


1. Сначала применяется свертка 1х1 для уменьшения количества каналов (чтобы свертка с размером ядра 3 применялась к изображению с меньшим количеством каналов, т.е. работала быстрее и с меньшим количеством параметров)
2. Далее делается основное преобразование: обычно это типичная последовательность действий `Conv -> BN -> Act`, не меняющая канальную размерность (она меняется при необходимости дальше из тех же соображений, что в п.1). В случае необходимости понизить пространственную размерность можно с помощью параметра `stride` у свертки.
3. Далее с помощью свертки 1х1 изображение приводится к желаемому количеству каналов.
4. В конце к выходу блока прибавляется вход (тот самый skip-connection). В случае если блок понижает пространственную размерность и повышает канальную, размерности исходного изображения и изображения после сверток не сходятся, поэтому перед сложением приходится применять дополнительное преобразование к исходной картинке. Его можно сделать с помощью все той же свертки 1х1 с параметром `stride`, соответствующим основному преобразованию.

#### Реализация сети

В ходе реализации сети вам могут помочь следующие рекомендации

Поскольку мы имеем дело с небольшими датасетами и небольшим количеством классов (и не собираемся доводить до 2048 каналов в конце), нет жесткой необходимости оптмизировать работу сверточных блоков. Вместо этого предлагается сделать менее эффективный, но более богатый параметрами блок. Можно использовать последовательность операций `Conv -> BN -> Act -> Conv -> BN` (перед сложением с "входом" блока лучше на ставить активацию), где одна из сверток выполняется со `stride=2`, если требуется. Также для облегчения обучения сети можно использовать свертку 1х1 (не забывайте нормализацию!) для преобразования входа, даже если увеличение количества каналов и уменьшение размерности не требуется.

1. Реализуйте отдельно ResidualBlock. Убедитесь, что он работает, как ожидается, и в случае сохранения размера изображения, и в случае его изменения. Для удобства мы даем вам шаблон. Не забудьте использовать `padding`.
2. Не забудьте сжать картинку перед применением ResidualBlock'ов с помощью свертки и пулинга. Степень сжатия зависит от размера вашего исходного изображения. Для размера изображения 256х256 можно попробовать сжать в 2-4 раза.
3. Global Average Pooling реализован в PyTorch классом `nn.AdaptiveAvgPool2d`. Для использования в нашей сети его можно добавить после последнего сверточного слоя следующим образом `nn.AdaptiveAvgPool2d((1, 1))`.
4. Вам не нужен ResNet50 :) Для проверки обучения начните с нескольких блоков, которые постепенно сжимают изображение - разжимают каналы в 2 раза. Тем не менее ваша сеть должна постепенно доводить канальную размерность не меньше, чем до 256 каналов.

Для начала реализуйте блок.

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        # последовательность операций самого блока
        self.blocks = nn.Sequential()
        # преобразование над входом до сложения с выходом
        self.downsample = nn.Sequential()

    def forward(self, x):
        # ваш код

Протестируйте блок.

In [ ]:
image_batch = torch.ones((3, 16, 67, 67))
block1 = ResidualBlock(16, 16, 32, 1)
assert block1(image_batch).shape == (3, 16, 67, 67), 'Блок не должен менять размер изображения'

In [ ]:
image_batch = torch.ones((3, 16, 64, 64))
block1 = ResidualBlock(16, 32, 32, 2)
assert block1(image_batch).shape == (3, 32, 32, 32), 'Блок должен сжать изображение в 2 раза и увеличить количество выходных каналов до 32'

Реализуйте сеть целиком.

In [ ]:
class ResNet(nn.Module):
    def __init__(self, num_classes, num_layers=None, layers_params=None):
        """
        Класс реализующий сеть типа ResNet

        param num_classes: количество классов, предсказываемых сетью
        param num_layers: список количества блоков с параметрами layers_params, которые будут добавлены в сеть
        param layers_params: список словарей параметров блоков
        """
        super().__init__()
        # количество классов для классификации
        self.num_classes = num_classes

        # дефолтные значения для примера
        if num_layers is None:
            num_layers = [1, 1, 1, 1, 1]
        if layers_params is None:
            layers_params = [
                {"in_channels": 16, "out_channels": 32, "stride": 2},
                {"in_channels": 32, "out_channels": 64, "stride": 2},
                {"in_channels": 64, "out_channels": 64, "stride": 1},
                {"in_channels": 64, "out_channels": 128, "stride": 2},
                {"in_channels": 128, "out_channels": 128, "stride": 1}
            ]
        
        assert len(num_layers) == len(layers_params), 'Размеры списков, задающих параметры сети, должны быть одинаковы'

        # Слои до residual блоков
        self.preprocess = nn.Sequential()

        # Задаем блоки с помощью num_layers и layers_params
        self.residual_blocks = nn.Sequential()

        # Голова сети
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        last_channels = # ваш код
        self.fc = nn.Linear(last_channels, num_classes)
        self.drop = nn.Dropout(0.1)

    def forward(self, x):
        # ваш код

Проверьте, что сеть работает

In [ ]:
resnet = ResNet(num_classes=4)
image_batch = torch.ones((3, 3, 36, 36))
resnet(image_batch).shape

In [ ]:
image_batch = torch.ones((3, 3, 132, 177))
resnet(image_batch).shape

### Обучение сети

**Важные советы:**

1. Если сеть с дефолтным значением lr начнет слишком нестабильно учиться, уменьшите его.
2. Используйте lr_scheduler для улучшения сходимости на более поздних итерациях. Можно, например, уменьшать lr в 2 раза после 3 эпох отсутствия улучшения метрики F1-macro.
3. Используйте параметр `weight_decay`, если сеть будет склонна к переобучению. Начать можно со значения 0.1.
4. Используйте аугментации для борьбы с переобучением. Отличный вариант - не слишком жесткие геометрические преобразования.


Напишите функцию для обучения сети, куда можно передавать аугментации и lr_scheduler в качестве параметров. Используйте их для экспериментов. Также ваша функция должна в качестве параметра принимать директорию, куда будут сохраняться чекпойнты сети по ходу обучения (можете сохранять после каждой эпохи, а можете - только лучший по метрике F1-macro). Создать директорию из функции можно с помощью `os.makedirs(checkpoints_dir)`, а путь к чейкпойнту задавать с помощью `os.path.join(checkpoints_dir, f'epoch_{epoch}.checkpoint')` (для случая сохранения после каждой эпохи). Для сохранения сети используйте `torch.save(model.state_dict(), PATH)`. 



**Ваша задача:** добиться метрики F1-macro > 60% на тестовом наборе. Опишите свои эксперименты.

In [ ]:
device = "cuda" if torch.cuda.is_available() else 'cpu'

In [ ]:
# ваш код

### II. Transfer learning

В этой части задания вам нужно зафайнтюнить предобученную модель, посчитать метрику на тестовом наборе и сравнить результат с предыдущем пунктом.

In [ ]:
# ваш код